Current columns in the dataset: ['text', 'original', 'labels', 'class_type', 'turker_gender', 'episode_done', 'confidence']

In [18]:
import pandas as pd # use for data manipulation and analysis
import numpy as np # use for multi-dimensional array and matrix

import seaborn as sns # use for high-level interface for drawing attractive and informative statistical graphics 
import matplotlib.pyplot as plt # It provides an object-oriented API for embedding plots into applications
%matplotlib inline 
# It sets the backend of matplotlib to the 'inline' backend:


from sklearn.naive_bayes import MultinomialNB # nlp algo use to Positive , Negative comment

from sklearn.model_selection import train_test_split # spliting the data between feature and target
from sklearn.metrics import classification_report # gives whole report about metrics (e.g, recall,precision,f1_score,c_m)
from sklearn.metrics import confusion_matrix # gives info about actual and predict
from nltk.tokenize import RegexpTokenizer # regexp tokenizers use to split words from text  
from nltk.stem.snowball import SnowballStemmer # stemmes words
from sklearn.feature_extraction.text import CountVectorizer # create sparse matrix of words using regexptokenizes  
from sklearn.pipeline import make_pipeline # use for combining all prerocessors techniuqes and algos

from PIL import Image # getting images in notebook
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator# creates words cloud


import pickle# use to dump model 

import warnings # ignores pink warnings 
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from datasets import load_dataset

# Load the Multi-Dimensional Gender Bias dataset from Hugging Face
dataset = load_dataset("md_gender_bias")


# Prepare data
data = pd.DataFrame(dataset["train"])
X = data["text"]
y = data["labels"]



2023-09-28 10:27:37.321726: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-09-28 10:27:37.321779: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-09-28 10:27:37.336265: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-28 10:27:38.539911: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 10:27:43.478967: W tensorflow/compiler/

In [3]:
#create a dataframe of classes counts
label_counts = pd.DataFrame(data.labels.value_counts())


In [4]:
from nltk.tokenize import RegexpTokenizer # regexp tokenizers use to split words from text
tokenizer = RegexpTokenizer(r'[A-Za-z]+')

In [5]:
data.text[0]

'He designed monumental Lovissa War Cemetery in 1920.'

In [6]:
print('Getting words tokenized ...')
data['text_tokenized'] = data.text.map(lambda t: tokenizer.tokenize(t)) # doing with all rows

Getting words tokenized ...


In [7]:
from nltk.stem.snowball import SnowballStemmer # stemmes words
stemmer = SnowballStemmer("english") # choose a language


In [8]:
print('Getting words stemmed ...')
data['text_stemmed'] = data['text_tokenized'].map(lambda l: [stemmer.stem(word) for word in l])

Getting words stemmed ...


In [9]:
data.sample(5)

,text,original,labels,class_type,turker_gender,episode_done,confidence,text_tokenized,text_stemmed
623,is the school structure intact till now Sir? I...,Is the school structure intact till now? i am ...,[3],1,0,True,certain,"[is, the, school, structure, intact, till, now...","[is, the, school, structur, intact, till, now,..."
1776,"she didnt catch it earlier, i cant get enough ...","she didnt catch it earlier, i cant get enough ...",[3],1,0,True,pretty sure,"[she, didnt, catch, it, earlier, i, cant, get,...","[she, didnt, catch, it, earlier, i, cant, get,..."
2341,"Cool, well I was born in France among pretty g...","Cool, well I was born in France, then I moved ...",[2],1,1,True,certain,"[Cool, well, I, was, born, in, France, among, ...","[cool, well, i, was, born, in, franc, among, p..."
1557,"in 2009, she also played the epic title told i...","In 2009, she also played the title told in Evi...",[0],0,0,True,certain,"[in, she, also, played, the, epic, title, told...","[in, she, also, play, the, epic, titl, told, i..."
828,"I like a broad spectrum of authors,. You do li...",I like a broad spectrum of authors. Do you lik...,[3],1,1,True,certain,"[I, like, a, broad, spectrum, of, authors, You...","[i, like, a, broad, spectrum, of, author, you,..."


In [13]:
data.isnull().sum()

text              0
original          0
labels            0
class_type        0
turker_gender     0
episode_done      0
confidence        0
text_tokenized    0
text_stemmed      0
text_sent         0
dtype: int64

In [10]:
print('Getting joiningwords ...')
data['text_sent'] = data['text_stemmed'].map(lambda l: ' '.join(l))

Getting joiningwords ...


In [16]:
# # sliceing classes
# if (data.labels == 0 ) or (data.labels == 0) :
#     negative_comment = data['text']
# elif  (data.labels == 5) or (data.labels == 4):
#     positive_comment = data['text']

# else :
#     neutral_comment = data['text']
# negative_comment = data[(data.labels == 0 ) or (data.labels == 0) ]
# positive_comment = data[(data.labels == 5) or (data.labels == 4)]
# neutral_comment= data[data.labels == 3]
# irrelevant_comment = data[data.sentiment == 'Irrelevant']

In [19]:
#create cv object
cv = CountVectorizer()

In [20]:
feature = cv.fit_transform(data.text_sent) #transform all text which we tokenize and stemmed

In [21]:
feature[:5].toarray() # convert sparse matrix into array to print transformed features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [22]:
trainX, testX, trainY, testY = train_test_split(feature, data.labels)  # spliting data

In [23]:
# create mnb object
mnb = MultinomialNB()

In [ ]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
mnb.fit(trainX,trainY)

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [ ]:


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# SVM Classifier
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

svm_classifier = SVC()
svm_classifier.fit(X_train_tfidf, y_train)
svm_predictions = svm_classifier.predict(X_test_tfidf)

print("SVM Classifier:")
print("Accuracy:", accuracy_score(y_test, svm_predictions))
print(classification_report(y_test, svm_predictions))

# Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train_tfidf, y_train)
rf_predictions = rf_classifier.predict(X_test_tfidf)

print("Random Forest Classifier:")
print("Accuracy:", accuracy_score(y_test, rf_predictions))
print(classification_report(y_test, rf_predictions))

# ANN with Hugging Face Transformers
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

ann_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
ann_predictions = ann_classifier(X_test.tolist(), truncation=True, padding=True)

ann_predictions = [entry['label'] for entry in ann_predictions]

print("Artificial Neural Network (BERT):")
print("Accuracy:", accuracy_score(y_test, ann_predictions))
print(classification_report(y_test, ann_predictions))

# RNN with PyTorch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn, optim
import torch

# Tokenize and encode text data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
encoded_texts = tokenizer(list(X_train), padding=True, truncation=True, return_tensors='pt')
X_train_tensors = encoded_texts['input_ids']
X_train_masks = encoded_texts['attention_mask']

encoded_texts = tokenizer(list(X_test), padding=True, truncation=True, return_tensors='pt')
X_test_tensors = encoded_texts['input_ids']
X_test_masks = encoded_texts['attention_mask']

# Create PyTorch DataLoader
batch_size = 32
train_data = TensorDataset(X_train_tensors, X_train_masks, torch.tensor(y_train.values))
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = TensorDataset(X_test_tensors, X_test_masks, torch.tensor(y_test.values))
test_loader = DataLoader(test_data, batch_size=batch_size)

# RNN Model
class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = outputs.logits
        return output

rnn_model = RNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(rnn_model.parameters(), lr=1e-5)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
rnn_model.to(device)
for epoch in range(3):
    rnn_model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = rnn_model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation
rnn_model.eval()
rnn_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, _ = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = rnn_model(input_ids, attention_mask)
        predictions = torch.argmax(outputs, dim=1).cpu().numpy()
        rnn_predictions.extend(predictions)

print("Recurrent Neural Network (BERT):")
print("Accuracy:", accuracy_score(y_test, rnn_predictions))
print(classification_report(y_test, rnn_predictions))


In [ ]:


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SVM Classifier
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

svm_classifier = SVC()
svm_classifier.fit(X_train_tfidf, y_train)
svm_predictions = svm_classifier.predict(X_test_tfidf)

print("SVM Classifier:")
print("Accuracy:", accuracy_score(y_test, svm_predictions))
print(classification_report(y_test, svm_predictions))

# Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train_tfidf, y_train)
rf_predictions = rf_classifier.predict(X_test_tfidf)

print("Random Forest Classifier:")
print("Accuracy:", accuracy_score(y_test, rf_predictions))
print(classification_report(y_test, rf_predictions))

# ANN with Hugging Face Transformers
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

ann_classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
ann_predictions = ann_classifier(X_test.tolist(), truncation=True, padding=True)

ann_predictions = [entry['label'] for entry in ann_predictions]

print("Artificial Neural Network (BERT):")
print("Accuracy:", accuracy_score(y_test, ann_predictions))
print(classification_report(y_test, ann_predictions))

# RNN with PyTorch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn, optim
import torch

# Tokenize and encode text data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
encoded_texts = tokenizer(list(X_train), padding=True, truncation=True, return_tensors='pt')
X_train_tensors = encoded_texts['input_ids']
X_train_masks = encoded_texts['attention_mask']

encoded_texts = tokenizer(list(X_test), padding=True, truncation=True, return_tensors='pt')
X_test_tensors = encoded_texts['input_ids']
X_test_masks = encoded_texts['attention_mask']

# Create PyTorch DataLoader
batch_size = 32
train_data = TensorDataset(X_train_tensors, X_train_masks, torch.tensor(y_train.values))
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = TensorDataset(X_test_tensors, X_test_masks, torch.tensor(y_test.values))
test_loader = DataLoader(test_data, batch_size=batch_size)

# RNN Model
class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = outputs.logits
        return output

rnn_model = RNNClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(rnn_model.parameters(), lr=1e-5)

# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
rnn_model.to(device)
for epoch in range(3):
    rnn_model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = rnn_model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation
rnn_model.eval()
rnn_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, _ = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = rnn_model(input_ids, attention_mask)
        predictions = torch.argmax(outputs, dim=1).cpu().numpy()
        rnn_predictions.extend(predictions)

print("Recurrent Neural Network (BERT):")
print("Accuracy:", accuracy_score(y_test, rnn_predictions))
print(classification_report(y_test, rnn_predictions))
